# Fine-tuning de LLM com o Dataset AmazonTitles-1.3MM

Este notebook demonstra o processo de fine-tuning de um modelo BERT usando o dataset AmazonTitles-1.3MM. O objetivo é treinar o modelo para gerar respostas baseadas em títulos e descrições de produtos da Amazon.

## 0. Instalação das Dependências

Primeiro, vamos instalar as bibliotecas necessárias para o projeto.

In [ ]:
!pip install transformers torch scikit-learn

## 1. Importação das Bibliotecas Necessárias

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

## 2. Preparação do Dataset

In [ ]:
class AmazonTitlesDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            item['title'],
            item['content'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

def load_data(file_path):
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

## 3. Configuração do Modelo e Tokenizador

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

## 4. Preparação dos Dados para Fine-tuning

In [ ]:
# Para o Google Colab, você pode fazer upload do arquivo ou usar um link para download
# Exemplo de upload:
from google.colab import files
uploaded = files.upload()

# Após o upload, use o nome do arquivo carregado
file_name = list(uploaded.keys())[0]
data = load_data(file_name)

train_data, val_data = train_test_split(data, test_size=0.1)

train_dataset = AmazonTitlesDataset(train_data, tokenizer, max_length=128)
val_dataset = AmazonTitlesDataset(val_data, tokenizer, max_length=128)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

## 5. Configuração do Otimizador

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

## 6. Loop de Treinamento

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validação
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            val_loss += outputs.loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss/len(val_dataloader)}")

## 7. Salvando o Modelo

In [ ]:
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

# Para baixar o modelo treinado do Colab
from google.colab import files
files.download('fine_tuned_model')

## 8. Função para Gerar Respostas

In [ ]:
def generate_response(title, content):
    inputs = tokenizer.encode_plus(title, content, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    
    # Aqui você pode mapear a classe prevista para uma resposta apropriada
    responses = ["Resposta negativa", "Resposta neutra", "Resposta positiva"]
    return responses[predicted_class]

# Exemplo de uso
title = "Smartphone de última geração"
content = "Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente."
response = generate_response(title, content)
print(f"Pergunta: {title}")
print(f"Resposta gerada: {response}")

## Conclusão

Este notebook demonstra o processo de fine-tuning de um modelo BERT usando o dataset AmazonTitles-1.3MM. O modelo treinado pode ser usado para gerar respostas baseadas em títulos e descrições de produtos.

Lembre-se de que este é um exemplo simplificado e pode ser aprimorado de várias maneiras:

1. Ajuste de hiperparâmetros para melhorar o desempenho.
2. Implementação de técnicas de regularização para evitar overfitting.
3. Uso de GPU para acelerar o treinamento (já habilitado no Colab).
4. Implementação de early stopping.
5. Utilização de técnicas mais avançadas de NLP para geração de respostas.

Para um projeto real, considere também implementar uma avaliação mais robusta do modelo e adaptar o código conforme necessário para atender aos requisitos específicos do projeto.